<a href="https://colab.research.google.com/github/ronykroy/DNN-NLP-and-other-stuff/blob/master/H20_multiClass_newsGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! apt-get install default-jre
!java -version
# install Java.. because.. there is nothing in this colab VM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [2]:
! pip install h2o
# H2O too to be installed SMH

     |████████████████████████████████| 123.6MB 120kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.8-py2.py3-none-any.whl size=123708939 sha256=0e8447da50c020c9bc962f43c6a8f02631c3f1acce2da9bd68ce749f0ad23bc9
  Stored in directory: /root/.cache/pip/wheels/f7/4c/2f/367de2996c0c2621a9e2e19624be0ea0e4b766df3cd597d916
Successfully built h2o


In [0]:
import h2o

In [4]:
h2o.init(max_mem_size="4G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfx407_ma
  JVM stdout: /tmp/tmpfx407_ma/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpfx407_ma/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_unknownUser_uwwmw3
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
df["Train"] = np.random.rand(len(df)) < 0.8 # get a col full of train boolean 

In [35]:
df.Train.value_counts()

True     9021
False    2293
Name: Train, dtype: int64

In [36]:
reviews = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train = reviews[reviews["Train"] == "True"]
test = reviews[reviews["Train"] == "False"]

In [38]:
train.head(2)

label,text,Train
17,"Although I realize that principle is not one of your strongest points, I would still like to know why do do not ask any question of this sort about the Arab countries. If you want to continue this think tank charade of yours, your fixation on Israel must stop. You might have to start asking the same sort of questions of Arab countries as well. You realize it would not work, as the Arab countries' treatment of Jews over the last several decades is so bad that your fixation on Israel would begin to look like the biased attack that it is. Everyone in this group recognizes that your stupid 'Center for Policy Research' is nothing more than a fancy name for some bigot who hates Israel.",True
15,"I read somewhere, I think in Morton Smith's _Jesus the Magician_, that old Lazarus wasn't dead, but going in the tomb was part of an initiation rite for a magi-cult, of which Jesus was also a part. It appears that a 3-day stay was normal. I wonder .... ?",True


In [0]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [0]:
# TODO: review how effective this is in removing junk \r \n etc
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [41]:
%%time
# Break reviews into sequence of words
words = tokenize(reviews["text"]) # not sure if there is some sort of leakage here...

CPU times: user 3.73 s, sys: 87.3 ms, total: 3.82 s
Wall time: 16.1 s


In [42]:
words.head()

C1
well
sure
story
nad
seem
biased
disagree
statement
media
ruin


In [43]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words) # train on the words thing we just got

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [44]:
# Sanity check - find synonyms for the word 'president'
w2v_model.find_synonyms("president", count = 5) # not bad actually :)

OrderedDict([('yeltsin', 0.8414702415466309),
             ('miyazawa', 0.8060656189918518),
             ('mubarak', 0.7901224493980408),
             ('stephanopoulos', 0.7726730108261108),
             ('dole', 0.7522024512290955)])

In [0]:
# Calculate a vector for each review
review_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [46]:
review_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.153198,0.00899349,-0.0483103,-0.104527,0.0873993,0.153018,0.0597841,0.0715517,0.0725264,-0.0560713,-0.0311968,0.0264233,0.0253788,0.0407529,-0.222762,-0.213541,-0.128641,-0.0974571,-0.0460212,0.0974529,0.0227679,0.108592,-0.144864,0.0722141,-0.000865031,-0.0427616,0.0267644,-0.1114,-0.0142342,-0.0600203,-0.0371627,0.168221,0.124309,0.00695359,-0.126992,0.0403636,0.0766551,-0.0602945,-0.148659,-0.244502,0.00262496,0.0205471,0.0355546,-0.0676994,0.0573237,-0.145366,0.105705,-0.0536737,0.0105741,0.0188049,-0.0683826,-0.122473,-0.0798966,0.0959842,-0.055041,0.0919155,0.0647284,0.0301453,-0.0267334,0.0240025,-0.15778,-0.173289,0.123475,-0.0869968,0.0658034,0.0864134,-0.0308045,-0.12271,0.0299651,-0.0900451,0.179367,-0.0669337,0.159203,0.0338325,-0.119192,-0.0945194,-0.0918812,0.145257,0.0181775,-0.021563,-0.076669,0.0893987,0.0967427,-0.0997966,0.0326413,0.174383,0.0840563,0.00190524,-0.0709991,0.0820513,0.0348251,-0.114836,-0.0377129,0.246896,0.0950981,0.0527845,0.0637647,0.0690961,-0.140496,0.115452
0.163248,0.0382057,-0.0198805,-0.0910298,0.0221363,-0.0228699,0.0821834,0.0269923,0.0965529,-0.0414153,-0.0466133,-0.0352459,0.0180831,0.0876958,-0.248039,-0.189693,-0.03633,0.00456666,0.000230621,0.0345671,0.0567957,0.0216514,-0.109361,-0.0180087,-0.0207769,-0.118431,0.0484506,-0.192853,0.0290335,0.0152954,0.0741835,-0.0269392,0.099133,0.0943563,-0.00657773,0.0582167,0.182341,-0.115106,-0.186124,-0.0863765,-0.0738247,0.0357499,0.0499714,0.0250119,-0.103973,-0.126641,-0.000106452,-0.112506,-0.0550739,0.0435841,-0.140376,-0.11736,-0.151796,0.0252546,0.0580068,0.0950189,2.58512e-05,0.00755062,0.0480986,0.122883,-0.0628134,-0.236853,-0.0940389,-0.025004,0.0113805,0.0310947,0.0334763,-0.156683,0.0623772,-0.0758292,0.083237,0.0608622,0.175095,0.0954334,-0.164587,-0.107304,-0.0444079,0.0681963,-0.0359948,-0.0024805,-0.0556648,0.122417,0.0817376,-0.120116,0.0637394,0.241502,0.0673772,-0.00404125,-0.0831525,0.0282735,0.0733905,-0.0372775,-0.0544624,0.135509,0.0566603,0.0144745,0.0748951,0.0442546,-0.000646352,0.0973201
0.160298,0.0803184,0.0241032,-0.0697197,0.0150393,0.186145,0.0794614,0.109135,0.0907112,-0.0376136,-0.0281645,-0.000439192,0.0593808,0.0451275,-0.268915,-0.226265,-0.0416185,-0.0702348,0.0202048,0.0490874,0.0168523,0.0396628,-0.0218766,0.0589169,0.0141875,-0.0511004,0.0473841,-0.103904,0.00824959,-0.0244149,-0.0295941,0.133062,0.198168,0.00721715,-0.0967421,0.0300974,0.0396138,-0.0112622,-0.0989613,-0.250019,-0.0456239,0.0384319,0.0762585,-0.070318,0.0262289,-0.124968,0.0587327,-0.106603,-0.0650594,0.0260054,-0.0361298,-0.0572146,-0.0539196,0.142825,-0.0536106,0.103129,-0.0166807,0.0344966,0.0135884,0.0500941,-0.0622015,-0.202263,0.0420565,-0.159986,0.0756524,0.106136,-0.0148756,-0.106691,-0.0374716,-0.125337,0.123111,-0.0752485,0.18684,0.091199,-0.15439,-0.106065,-0.0939722,0.169286,0.00466799,-0.0266638,-0.0242668,0.105259,0.0554042,-0.108179,0.086681,0.252609,0.0667523,0.00820405,-0.047489,0.0420691,0.063162,-0.135979,-0.0401593,0.231098,0.0772157,0.020158,0.0553891,0.0782379,-0.112158,0.0934862
0.155341,0.0933737,-0.0316468,0.0679775,0.0181642,0.0705945,0.096708,0.0785072,0.110272,-0.0547936,-0.00459857,-0.0798787,0.0496482,0.0194512,-0.248633,-0.22191,0.0346633,-0.0270729,-0.0239565,0.122555,0.0869296,0.050059,-0.138955,0.0399704,-0.0014212,0.0564061,-0.00563521,-0.172987,-0.0160485,-0.0468536,-0.0151299,-0.0394243,0.126508,0.013347,-0.0215226,0.0491002,0.0984097,0.0883454,-0.201989,-0.100941,-0.061561,-0.00747938,0.0425562,-0.0407724,0.00191125,-0.11774,0.0191509,-0.0928288,-0.0143992,0.0260623,0.0473006,-0.0721613,0.00440518,0.0612213

In [0]:
# Add aggregated word embeddings 
ext_reviews = reviews.cbind(review_vecs)
# rename may be... all 'reviews' to 'news'
# ext becaus ewe are extending the reviews df with the vectors

In [0]:
ext_reviews["label_f"]  = ext_reviews["label"] .asfactor()

In [0]:
# Splitting to test train.. along with the review vecs is difficult without the 'Train' boolean column
# hence redoing it from that stage..
# so that we'll be able to do this following cell

In [0]:
ext_train = ext_reviews[ext_reviews["Train"] == "True"]
ext_test = ext_reviews[ext_reviews["Train"] == "True"]

In [0]:
#review_vecs.names
# had a 100 cols
# may be more would give a better model ..?

In [0]:
predictors =  review_vecs.names

In [0]:
response = 'label_f'

In [0]:
from h2o.estimators import H2OGradientBoostingEstimator

In [77]:
gbm_embeddings = H2OGradientBoostingEstimator(stopping_metric = "AUTO", stopping_tolerance = 0.001,
                                              stopping_rounds = 5, score_tree_interval = 10,
                                              model_id = "gbm_embeddings.hex"
                                             )
gbm_embeddings.train(x = predictors, y = response, 
                   training_frame = ext_train, validation_frame = ext_test
                  )
# stopping metric was auc

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [78]:
gbm_embeddings.score_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2019-10-23 13:05:50,0.007 sec,0.0,0.954545,3.091042,0.948121,0.954545,3.091042,0.948121
1,,2019-10-23 13:06:09,18.758 sec,10.0,0.629752,1.143171,0.203636,0.629752,1.143171,0.203636
2,,2019-10-23 13:06:25,34.974 sec,20.0,0.488101,0.716244,0.124820,0.488101,0.716244,0.124820
3,,2019-10-23 13:06:41,50.838 sec,30.0,0.392415,0.489401,0.074160,0.392415,0.489401,0.074160
4,,2019-10-23 13:06:56,1 min 6.002 sec,40.0,0.319667,0.351744,0.046558,0.319667,0.351744,0.046558
5,,2019-10-23 13:07:12,1 min 21.406 sec,50.0,0.262849,0.262649,0.033367,0.262849,0.262649,0.033367


In [81]:
gbm_embeddings.confusion_matrix(ext_test)


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,115,460,Error,Rate
0,365.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.039474,15 / 380
1,2.0,435.0,0.0,0.0,1.0,1.0,0.0,14.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046053,21 / 456
2,1.0,1.0,449.0,1.0,3.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054737,26 / 475
3,0.0,2.0,0.0,464.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031315,15 / 479
4,2.0,0.0,0.0,0.0,442.0,0.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.049462,23 / 465
5,2.0,0.0,0.0,0.0,0.0,474.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006289,3 / 477
6,0.0,1.0,0.0,1.0,1.0,0.0,453.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021598,10 / 463
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,468.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.014737,7 / 475
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,461.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.033543,16 / 477
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,23.0,0.0,450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050633,24 / 474



See the whole table with table.as_data_frame()


In [82]:
gbm_embeddings.cross_validation_metrics_summary

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_embeddings.hex


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,1000.0,406878.0,5.0,5.0,5.0,17.0,32.0,27.655




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.0690898359902942
RMSE: 0.26284945499333684
LogLoss: 0.26264853071954264
Mean Per-Class Error: 0.03088453582602321

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,115,460,Error,Rate
0,365.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.039474,15 / 380
1,2.0,435.0,0.0,0.0,1.0,1.0,0.0,14.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046053,21 / 456
2,1.0,1.0,449.0,1.0,3.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054737,26 / 475
3,0.0,2.0,0.0,464.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031315,15 / 479
4,2.0,0.0,0.0,0.0,442.0,0.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.049462,23 / 465
5,2.0,0.0,0.0,0.0,0.0,474.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006289,3 / 477
6,0.0,1.0,0.0,1.0,1.0,0.0,453.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021598,10 / 463
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,468.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.014737,7 / 475
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,461.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.033543,16 / 477
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,23.0,0.0,450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050633,24 / 474



See the whole table with table.as_data_frame()

Top-10 Hit Ratios: 


...

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2019-10-23 13:05:50,0.007 sec,0.0,0.954545,3.091042,0.948121,0.954545,3.091042,0.948121
1,,2019-10-23 13:06:09,18.758 sec,10.0,0.629752,1.143171,0.203636,0.629752,1.143171,0.203636
2,,2019-10-23 13:06:25,34.974 sec,20.0,0.488101,0.716244,0.124820,0.488101,0.716244,0.124820
3,,2019-10-23 13:06:41,50.838 sec,30.0,0.392415,0.489401,0.074160,0.392415,0.489401,0.074160
4,,2019-10-23 13:06:56,1 min 6.002 sec,40.0,0.319667,0.351744,0.046558,0.319667,0.351744,0.046558
5,,2019-10-23 13:07:12,1 min 21.406 sec,50.0,0.262849,0.262649,0.033367,0.262849,0.262649,0.033367



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C98,950.125122,1.000000,0.043798
1,C26,752.297791,0.791788,0.034679
2,C51,740.954529,0.779849,0.034156
3,C7,680.540283,0.716264,0.031371
4,C63,669.093994,0.704217,0.030844
5,C39,585.124939,0.615840,0.026973
6,C25,513.204102,0.540144,0.023657
7,C15,460.721497,0.484906,0.021238
8,C64,456.620270,0.480590,0.021049
9,C61,412.500031,0.434153,0.019015



See the whole table with table.as_data_frame()


<bound method ModelBase.cross_validation_metrics_summary of >

In [0]:
# stacked ensembles
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/base_models.html?highlight=h2ogradientboostingestimator